<h1 align="center">ALGORYTM DETEKCJI NOWOTWORU SKÓRY</h1>
<h3 align="center">Etap 1 - wykrywanie krawędzi zmiany</h3>
<br>
<p>Serwis dostępny na: [melanoma.tk](http://melanoma.tk)</p>
<br>    

In [24]:
import os
import glob
import random
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [25]:
imagePath = os.getcwd() + '/uploads/'
histogramPath = os.getcwd() + '/static/histograms/'
filename = "melanomaCancer.jpg"

<h1 align="center">Krok 0: Pobranie od użytkownika obrazu</h1>
<br>
<p>Użytkownik "uploaduje" obraz swojej zmiany skóry:</p> 
<br>
<img src="static/melanomaCancer.jpg" align="center" width=50%>
<br>
<p>Celem algorytmu jest to aby po przetworzeniu obrazu zwrócił prawdopodobieństwo, z jakim ta zmiana jest nowotworem.</p>
<br>

In [26]:
# Step 1 read convert to grayscale image
img = cv2.imread(imagePath + filename, cv2.IMREAD_GRAYSCALE)
# save result
cv2.imwrite(histogramPath + "0Started.jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

True

<h1 align="center">Krok 1: Wczytanie obrazu i konwersja do skali szarości</h1>
<br>
<img src="static/histograms/0Started.jpg" width=50%>
<br>
<p>Obraz jest wczytywany przy pomocy metody imread() z biblioteki opencv (cv2) z parametrem IMREAD_GRAYSCALE co oznacza konwersję wczytanego kolorowego obrazu do obrazu w skali szarości.</p>
<br>

In [27]:
# Step 2: Gaussian filter
denoised = cv2.GaussianBlur(img, (5, 5), 0)
# save result
cv2.imwrite(histogramPath + "1denoised.jpg", denoised, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

True

<h1 align="center">Krok 2: Filtr Gaussa</h1>
<br>
<img src="static/histograms/1denoised.jpg" width=50%>
<br>
<p>Obraz jest wygładzany przy pomocy dolnoprzepustowego filtru Gaussa.</p>
<br>

In [28]:
#Step 3: Thresholding
ret, thresh2 = cv2.threshold(denoised, 127, 255, cv2.THRESH_BINARY_INV)
# save result
cv2.imwrite(histogramPath + "2Tresh.jpg", thresh2, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

True

<h1 align="center">Krok 3: Thresholding - binaryzacja</h1>
<br>
<img src="static/histograms/2Tresh.jpg" width=50%>
<br>
<p>Obraz jest poddawany binaryzacji z progiem 127 powyżej, którego pixelom przypisywana jest wartość 0, a poniżej wartość 1.
<br>

In [29]:
denoisedTresh = cv2.GaussianBlur(thresh2, (5, 5), 0)
# save result
cv2.imwrite(histogramPath + "3denoisedThresh.jpg", denoisedTresh, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

True

<h1 align="center">Krok 4: Filtr Gaussa</h1>
<br>
<img src="static/histograms/3denoisedThresh.jpg" width=50%>
<br>
<p>Zbinaryzowany obraz jest poddawany kolejnej filtracji Gaussa w celu wygładzenia krawędzi "dziór", które będą w następnym kroku wypełniane.</p>
<br>

In [30]:
flood_fill = denoisedTresh.copy()

In [31]:
h, w = denoisedTresh.shape[:2]
mask = np.zeros((h + 2, w + 2), np.uint8)

# Step 5: Floodfill from point (0, 0)
cv2.floodFill(flood_fill, mask, (0, 0), 255)

(351992, array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=uint8), (0, 0, 920, 613))

In [32]:
# Invert floodfilled image
flood_fill_inv = cv2.bitwise_not(flood_fill)
# save result
cv2.imwrite(histogramPath + "5invOut.jpg", flood_fill_inv, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

True

<h1 align="center">Krok 5: Wytworzenie maski</h1>
<br>
<img src="static/histograms/5invOut.jpg" width=50%>
<br>
<p>Wytworzenie maski czarnych dziór wewnątrz obiektu zmiany, która w następnej operacji logicznej wypełni cały obszar zmiany.</p>
<br>

In [33]:
# Combine the two images to get the foreground.
filled_out = denoisedTresh | flood_fill_inv
# save result
cv2.imwrite(histogramPath + "6Out.jpg", filled_out, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

True

<h1 align="center">Krok 6: Wypełnienie obszaru zmiany</h1>
<br>
<img src="static/histograms/6Out.jpg" width=50%>
<br>
<p>Obraz po operacji OR na masce i obrazie z kroku 4 jest już zmianą która w całości jest wypełniona, dzoęki czemu możemy teraz łatwo wykryć krawędzie.</p>
<br>

In [34]:
# Step 6: Detect edges - Laplacian
filter2 = cv2.Laplacian(filled_out, cv2.CV_64F)
# save result
cv2.imwrite(histogramPath + "7detectedEdges.jpg", filter2, [int(cv2.IMWRITE_JPEG_QUALITY), 95]) 

True

![]("histograms/7detectedEdges.jpg")

<h1 align="center">Krok 7: Laplacjan</h1>
<br>
<img src="static/histograms/7detectedEdges.jpg" width=50%>
<br>
<p>Wykonanie wyostrzania Laplacjanu pozwoliło na eleganckie wyznaczenie krawędzi zmiany skórnej, co jest bardzo ważne do dalszych kroków wyliczania prawdopodobieństwa nowotworowości zmiany opartych na symetri i regularności zmiany skórnej.</p>
<br>